# Setup


In [ ]:
!pip install ibm-cos-sdk==2.6 python-dotenv

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

bucket = os.getenv("BUCKET", "")
access_key_id = os.getenv("ACCESS_KEY_ID", "")
secret_access_key = os.getenv("SECRET_ACCESS_KEY", "")
endpoint_url = os.getenv("ENDPOINT_URL", "")

In [ ]:
import os
import shutil

if os.path.exists('tmp') and os.path.isdir('tmp'):
    shutil.rmtree('tmp')

if os.path.exists('model_web') and os.path.isdir('model_web'):
    shutil.rmtree('model_web')

CLOUD_ANNOTATIONS_DATA = os.path.join('tmp', bucket)

os.makedirs(CLOUD_ANNOTATIONS_DATA, exist_ok=True)


In [ ]:
import json
import ibm_boto3

def download_file_cos(local_file_name, key): 
    '''
    Wrapper function to download a file from cloud object storage using the
    credential dict provided and loading it into memory
    '''
    cos = ibm_boto3.client(
        service_name='s3',
        aws_access_key_id=access_key_id,
        aws_secret_access_key=secret_access_key,
        endpoint_url=endpoint_url)
    try:
        res=cos.download_file(Bucket=bucket, Key=key, Filename=local_file_name)
    except Exception as e:
        print('Exception', e)
    else:
        print('File Downloaded')

def get_annotations(): 
    cos = ibm_boto3.client(
        service_name='s3',
        aws_access_key_id=access_key_id,
        aws_secret_access_key=secret_access_key,
        endpoint_url=endpoint_url)
    try:
        return json.loads(cos.get_object(Bucket=bucket, Key='_annotations.json')['Body'].read())
    except Exception as e:
        print('Exception', e)

In [ ]:
annotations = get_annotations()

download_file_cos(os.path.join(CLOUD_ANNOTATIONS_DATA, '_annotations.json'), '_annotations.json')

for image in annotations['annotations'].keys():
    local_path = os.path.join(CLOUD_ANNOTATIONS_DATA, image)
    download_file_cos(local_path, image)